In [ ]:
%pip install 

In [2]:
from llama_index import SimpleDirectoryReader, StorageContext, ServiceContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_iris import IRISVectorStore

In [4]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

#os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


In [18]:
documents = SimpleDirectoryReader("data/healthcare/notes").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: b6f9ef34-4c1b-456c-be9d-ad4dc8223eda


In [15]:
username = 'demo'
password = 'demo' 
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '65066' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [16]:
vector_store = IRISVectorStore.from_params(
    connection_string=CONNECTION_STRING,
    table_name="augmented_notes",
    embed_dim=1536,  # openai embedding dimension
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model, llm=None
# )

In [19]:
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    show_progress=True, 
    # service_context=service_context,
)
query_engine = index.as_query_engine()

Generating embeddings: 100%|██████████| 21/21 [00:01<00:00, 17.82it/s]


In [7]:
# # If reconnecting to the vector store, use this: 

# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# query_engine = index.as_query_engine()

# # Adding documents to existing index

# for d in documents:
#     index.insert(document=d, storage_context=storage_context)

In [24]:
response = query_engine.query("where did the patient have pain and discomfort")

In [25]:
import textwrap
print(textwrap.fill(str(response), 100))

The patient experienced pain and discomfort in the left proximal forearm after a fall.


In [10]:
response = query_engine.query("What happened in the mid 1980s?")
print(textwrap.fill(str(response), 100))

AI was in the air in the mid 1980s.
